# Geolocalização de CEPs

In [ ]:
# Bases OSM: https://download.geofabrik.de/south-america/brazil.html
### DOC: https://download.geofabrik.de/osm-data-in-gis-formats-free.pdf
### Extraidas apenas 'roads', 'places' e 'pois'

# Bases IBGE:
### https://www.ibge.gov.br/geociencias/downloads-geociencias.html
### organizacao_do_territorio > analises_do_territorio > divisao_regional > 
### divisao_regional_do_brasil > divisao_regional_do_brasil_em_regioes_geograficas_2017 > shp > 
### RG2017_regioesgeograficas2017_20180911.zip

### https://www.ibge.gov.br/geociencias/downloads-geociencias.html
### estrutura_territorial > localidades > Shapefile_SHP >
### BR_Localidades_2010_v1.dbf
### BR_Localidades_2010_v1.prj
### BR_Localidades_2010_v1.shp
### BR_Localidades_2010_v1.shx

### ftp://ftp.ibge.gov.br/Censos/Censo_Demografico_2010/Cadastro_Nacional_de_Enderecos_Fins_Estatisticos
### Cadastro Nacional de Enderecos Fins Estatisticos - um por estado - extraidos e renomeados conforme nome do estado
### DOC: https://biblioteca.ibge.gov.br/index.php/biblioteca-catalogo?view=detalhes&id=2101639

### Definição de variáveis globais e funções

In [1]:
# Import das bibliotecas:
import csv, os, re
from collections import OrderedDict
import xml.etree.ElementTree as ET
import fiona
import unicodedata
import datetime
import pandas as pd

In [2]:
# Retorna a data hora atual:
def get_hora_atual():
    return str(datetime.datetime.now())[:19]

In [3]:
# Percorre shapefile e imprime a linha solicitada:
def print_linha_shapeFile(arquivo_origem, numero_linha):
    origem = fiona.open(arquivo_origem)
    limite = len(origem)
    if (numero_linha > 0 and numero_linha < limite):
        linha_atual = 0
        for linha in origem:
            if (linha_atual == numero_linha):
                return linha
            if (linha_atual >= limite):
                print('Linha inexistente. Informe um valor menor e superior a zero.')
                break
            linha_atual += 1
    else:
        print('Linha inexistente. Informe um valor menor e superior a zero.')
    origem.close()


In [4]:
# Extrai variaveis do shapefile, a partir do dicionario informado:
def extrai_variaveis_shapeFile(arquivo_origem, arquivo_destino, numero_linhas, dicionario_variaveis, encoding='ascii'):
    # MODELO DE DICIONARIO:
    # Os campos devem ficar na chave, separados por virgula
    # O valor deve ser um booleano que indica se o campo eh nullable
    #     dicionario_variaveis_shape = {
    #         "properties.name": False, 
    #         "geometry.coordinates": False,
    #         "properties.fclass": True,
    #         "properties.ref": True
    #     }
    lista_campos = list(dicionario_variaveis.keys())
    campos = '|'.join(lista_campos) + '\n'
    origem = fiona.open(arquivo_origem)
    destino = open(arquivo_destino, 'w+', newline='', encoding="utf-8") 
    destino.write(campos)
    if (numero_linhas == -1):
        limite = len(origem)
    else:
        limite = numero_linhas
    print("Total de linhas a serem lidas do arquivo {0}: {1}.\nExtração iniciada às {2}."\
          .format(arquivo_origem, limite, get_hora_atual()))
    linha_atual = 0
    total_linhas = 0
    linha_destino_anterior = []
    for linha in origem:
        linha_destino = []
        valor_principal_nulo = False
        for k, v in dicionario_variaveis.items():
            valor = ''
            aux_k = linha
            k_encontrado = False
            for k1 in k.split('.'):
                if k1 in aux_k.keys():
                    aux_k = aux_k[k1]
                    k_encontrado = True
                else:
                    k_encontrado = False
                    break
            if (k_encontrado and str(aux_k) != 'None'):
                if (encoding != 'ascii'):
                    aux_k = str(aux_k).encode(encoding).decode('utf-8')
                valor = re.sub(r'\|', '', unicodedata.normalize('NFKD', str(aux_k)))\
                    .encode('ascii', 'ignore').decode('utf-8').upper().strip()
            if (v == False and valor == ''):
                valor_principal_nulo = True
                break
            linha_destino.append(valor)
        if (valor_principal_nulo == False):
            if (linha_destino_anterior != linha_destino):
                destino.write('|'.join(linha_destino) + '\n')
                total_linhas +=1
            linha_destino_anterior = linha_destino
        if (linha_atual % 100 == 0):
            progresso = int(100*linha_atual/limite)
            print('Percentual de registros gravados no CSV destino: %d%%\r'%progresso, end="")
        if (linha_atual >= limite):
            break
        linha_atual += 1
    print("Total de linhas gravadas em {0}: {1}.\n Fim da extração às {2}"\
          .format(arquivo_destino, total_linhas, get_hora_atual()))
    origem.close()
    destino.close()


In [5]:
# Extrai variaves a partir de um arquivo posicional, conforme dicionario de variaveis informado:
def extrai_variaveis_arquivo_posicional(arquivo_origem, arquivo_destino, numero_linhas, dicionario_variaveis):
    lista_campos = list(dicionario_variaveis.keys())
    campos = '|'.join(lista_campos) + '\n'
    origem = open(arquivo_origem, 'r')
    destino = open(arquivo_destino, 'w+', newline='', encoding="utf-8") 
    destino.write(campos)
    if (numero_linhas == -1):
        aux_origem = open(arquivo_origem, 'r')
        limite = sum(1 for l in aux_origem)
        aux_origem.close()
    else:
        limite = numero_linhas
    print("Total de linhas a serem lidas do arquivo {0}: {1}.\nExtração iniciada às {2}."\
          .format(arquivo_origem, limite, get_hora_atual()))
    linha_atual = 0
    total_linhas = 0
    linha_destino_anterior = []
    for linha in origem:
        linha_destino = []
        for k, v in dicionario_variaveis.items():
            valor = re.sub(r'\|', '', unicodedata.normalize('NFKD', linha[v[0]:v[1]]))\
                .encode('ascii', 'ignore').decode('utf-8').upper().strip()
            linha_destino.append(valor)
        if (linha_destino_anterior != linha_destino):
            destino.write('|'.join(linha_destino) + '\n')
            total_linhas +=1
        linha_destino_anterior = linha_destino
        if (linha_atual % 100 == 0):
            progresso = int(100*linha_atual/limite)
            print('Percentual de registros gravados no CSV destino: %d%%\r'%progresso, end="")
        if (linha_atual >= limite):
            break
        linha_atual += 1
    print("Total de linhas gravadas em {0}: {1}.\n Fim da extração às {2}"\
          .format(arquivo_destino, total_linhas, get_hora_atual()))
    origem.close()
    destino.close()


In [6]:
# Dataframe com codigo da UF segundo IBGE, sigla da UF e regiao:
df_ufs = pd.DataFrame([
    [11, 'RO', 'norte'],
    [12, 'AC', 'norte'],
    [13, 'AM', 'norte'],
    [14, 'RR', 'norte'],
    [15, 'PA', 'norte'],
    [16, 'AP', 'norte'],
    [17, 'TO', 'norte'],
    [21, 'MA', 'nordeste'],
    [22, 'PI', 'nordeste'],
    [23, 'CE', 'nordeste'],
    [24, 'RN', 'nordeste'],
    [25, 'PB', 'nordeste'],
    [26, 'PE', 'nordeste'],
    [27, 'AL', 'nordeste'],
    [28, 'SE', 'nordeste'],
    [29, 'BA', 'nordeste'],
    [31, 'MG', 'sudeste'],
    [32, 'ES', 'sudeste'],
    [33, 'RJ', 'sudeste'],
    [35, 'SP', 'sudeste'],
    [41, 'PR', 'sul'],
    [42, 'SC', 'sul'],
    [43, 'RS', 'sul'],
    [50, 'MS', 'centro-oeste'],
    [51, 'MT', 'centro-oeste'],
    [52, 'GO', 'centro-oeste'],
    [53, 'DF', 'centro-oeste']
], columns=['codigo', 'sigla', 'regiao'])

---

### Estudo e extração dos arquivos

#### 1. OpenStreetMap (© contribuidores do OpenStreetMap - dados disponíveis sob a Open Database License (ODbL))

Extração de locais e logradouros.

In [18]:
# Exemplo de linha extraida de ShapeFile Roads da Regiao Centro-Oeste, arquivo do OSM:
print_linha_shapeFile('data/shapefiles/centro-oeste/gis_osm_roads_free_1.shp', 5)

{'type': 'Feature',
 'id': '5',
 'geometry': {'type': 'LineString',
  'coordinates': [(-47.9272013, -15.8452793),
   (-47.9277525, -15.8451682),
   (-47.9280343, -15.8450709),
   (-47.9282637, -15.8449333),
   (-47.929314, -15.8440503),
   (-47.9307061, -15.8431258),
   (-47.9308938, -15.8429529),
   (-47.9309917, -15.8428136),
   (-47.931158, -15.8424601),
   (-47.9312171, -15.8422498)]},
 'properties': OrderedDict([('osm_id', '10064602'),
              ('code', 5132),
              ('fclass', 'trunk_link'),
              ('name', None),
              ('ref', None),
              ('oneway', 'F'),
              ('maxspeed', 0),
              ('layer', 0),
              ('bridge', 'F'),
              ('tunnel', 'F')])}

In [196]:
# Lista regioes e tipos de shapefile:
regioes = ['centro-oeste', 'nordeste', 'norte', 'sudeste', 'sul']
tipos_shape = ['roads', 'places', 'pois']

# Define dicionario para aplicacao na funcao:
# O booleano indica se eh nullable:
dicionario_variaveis_shape = {
    "properties.name": False, 
    "geometry.coordinates": False,
    "properties.fclass": True,
    "properties.ref": True
}

# Trata arquivo em loop por todas as regioes e tipos de shapefile:
numero_linhas = -1
for regiao in regioes:
    for tipo_shape in tipos_shape:
        arquivo_origem = "data/shapefiles/{0}/gis_osm_{1}_free_1.shp".format(regiao, tipo_shape)
        arquivo_destino = "data/shapefiles/tratados/{0}_{1}.csv".format(regiao, tipo_shape)
        print("\nExtraindo arquivo do tipo {0} da região '{1}'...".format(tipo_shape, regiao))
        extrai_variaveis_shapeFile(arquivo_origem, arquivo_destino, numero_linhas, dicionario_variaveis_shape)

print("\nFim das extrações.")


Extraindo arquivo do tipo roads da região 'centro-oeste'...
Total de linhas a serem lidas do arquivo C:/Users/Luiz/Documents/Python/geo/data/shapefiles/centro-oeste/gis_osm_roads_free_1.shp: 430315.
Extração iniciada às 2020-07-16 12:26:10.
Total de linhas gravadas em C:/Users/Luiz/Documents/Python/geo/data/shapefiles/tratados/centro-oeste_roads.csv: 179506.
 Fim da extração às 2020-07-16 12:26:35

Extraindo arquivo do tipo places da região 'centro-oeste'...
Total de linhas a serem lidas do arquivo C:/Users/Luiz/Documents/Python/geo/data/shapefiles/centro-oeste/gis_osm_places_free_1.shp: 6452.
Extração iniciada às 2020-07-16 12:26:35.
Total de linhas gravadas em C:/Users/Luiz/Documents/Python/geo/data/shapefiles/tratados/centro-oeste_places.csv: 6203.
 Fim da extração às 2020-07-16 12:26:36

Extraindo arquivo do tipo pois da região 'centro-oeste'...
Total de linhas a serem lidas do arquivo C:/Users/Luiz/Documents/Python/geo/data/shapefiles/centro-oeste/gis_osm_pois_free_1.shp: 17243.


#### 2. IBGE

Extração dos dados do IBGE sobre municípios e códigos regionais.

In [257]:
# Exemplo de linha extraida de ShapeFile de Regioes Geograficas do IBGE:
print_linha_shapeFile('data/shapefiles/brasil/divisao_urbano_regional/RG2017_regioesgeograficas2017.shp', 1)

{'type': 'Feature',
 'id': '1',
 'geometry': {'type': 'Polygon',
  'coordinates': [[(-48.867003127999965, -15.097346905999927, 0.0),
    (-48.86643752899994, -15.096298427999955, 0.0),
    (-48.86457896099995, -15.094616429999974, 0.0),
    (-48.86399747699994, -15.093262286999959, 0.0),
    (-48.863029145999974, -15.092169584999965, 0.0),
    (-48.861913062999974, -15.091803030999927, 0.0),
    (-48.86122319699996, -15.092669126999965, 0.0),
    (-48.861057940999956, -15.09361912999998, 0.0),
    (-48.859806041999946, -15.095798195999976, 0.0),
    (-48.85864126599995, -15.095906937999928, 0.0),
    (-48.85727688999998, -15.095379837999928, 0.0),
    (-48.856912498999975, -15.094246744999964, 0.0),
    (-48.85594301899994, -15.092431716999954, 0.0),
    (-48.85655317699997, -15.089123219999976, 0.0),
    (-48.85621763599994, -15.08826006199996, 0.0),
    (-48.85538329299993, -15.087825108999937, 0.0),
    (-48.854305444999966, -15.087803420999933, 0.0),
    (-48.853383950999955, -15.0

In [261]:
# Define dicionario para aplicacao na funcao:
dicionario_variaveis_shape = {
    "properties.NOME": False, 
    "properties.CD_GEOCODI": False, 
    "properties.UF": False, 
    "geometry.coordinates": False
}
numero_linhas = -1
arquivo_origem = "data/shapefiles/brasil/divisao_urbano_regional/RG2017_regioesgeograficas2017.shp"
arquivo_destino = "data/shapefiles/tratados/areas_municipios.csv"
print("\nExtraindo arquivo do tipo {0} da região '{1}'...".format(tipo_shape, regiao))
extrai_variaveis_shapeFile(arquivo_origem, arquivo_destino, numero_linhas, dicionario_variaveis_shape, 'latin1')

print("\nFim das extrações.")
# Aproximadamente 30 minutos para todas as tabelas


Extraindo arquivo do tipo pois da região 'sul'...
Total de linhas a serem lidas do arquivo C:/Users/Luiz/Documents/Python/geo/data/shapefiles/brasil/divisao_urbano_regional/RG2017_regioesgeograficas2017.shp: 5570.
Extração iniciada às 2020-07-16 14:13:26.
Total de linhas gravadas em C:/Users/Luiz/Documents/Python/geo/data/shapefiles/tratados/areas_municipios.csv: 5570.
 Fim da extração às 2020-07-16 14:14:14

Fim das extrações.


#### 3. Censo 2010

In [102]:
# Define dicionario para aplicacao na funcao:
dicionario_variaveis_cnefe = {
    "codigo_completo_setor": [0, 14],
    "nome_logradouro": [66, 125],
    "cep": [550, 558]
}
# Lista UFs:
ufs = [ 'AC','AL','AM','AP','BA',
        'CE','DF','ES','GO','MA',
        'MG','MS','MT','PA','PB',
        'PE','PI','PR','RJ','RN',
        'RO','RR','RS','SC','SE',
        'SP','TO']

# Trata arquivo em loop por todas as UFs:
numero_linhas = -1
for uf in ufs:
    arquivo_origem = "data/censo2010/enderecos_fins_estatisticos/{0}.txt".format(uf)
    arquivo_destino = "data/censo2010/enderecos_fins_estatisticos/tratados/{0}.csv".format(uf)
    print("Extraindo arquivo da UF '{0}'...".format(uf))
    extrai_variaveis_arquivo_posicional(arquivo_origem, arquivo_destino, numero_linhas, dicionario_variaveis_cnefe)

print("Fim das extrações.")

Extraindo arquivo da UF 'AC'...
Total de linhas a serem lidas do arquivo C:/Users/Luiz/Documents/Python/geo/data/censo2010/enderecos_fins_estatisticos/AC.txt: 283321.
Extração iniciada.
Total de linhas gravadas em C:/Users/Luiz/Documents/Python/geo/data/censo2010/enderecos_fins_estatisticos/tratados/AC.csv: 15186.

Extraindo arquivo da UF 'AL'...
Total de linhas a serem lidas do arquivo C:/Users/Luiz/Documents/Python/geo/data/censo2010/enderecos_fins_estatisticos/AL.txt: 1208310.
Extração iniciada.
Total de linhas gravadas em C:/Users/Luiz/Documents/Python/geo/data/censo2010/enderecos_fins_estatisticos/tratados/AL.csv: 56267.

Extraindo arquivo da UF 'AM'...
Total de linhas a serem lidas do arquivo C:/Users/Luiz/Documents/Python/geo/data/censo2010/enderecos_fins_estatisticos/AM.txt: 1091726.
Extração iniciada.
Total de linhas gravadas em C:/Users/Luiz/Documents/Python/geo/data/censo2010/enderecos_fins_estatisticos/tratados/AM.csv: 63657.

Extraindo arquivo da UF 'AP'...
Total de linhas

#### 4. Verificação

Conta a quantidade de CEPs distintos encontrados nas bases do Censo 2010.

In [33]:
qt_ceps_distintos = 0

for uf in ufs:
    arquivo_origem = "C:/Users/Luiz/Documents/Python/geo/data/censo2010/enderecos_fins_estatisticos/tratados/{0}.csv".format(uf)
    qt_ceps_distintos += len(pd.read_csv(arquivo_origem, sep='|')['cep'].unique())
    
print("Quantidade total de CEPs encontrados nas tabelas do Censo 2010: {0}".format(qt_ceps_distintos))

Quantidade total de CEPs encontrados nas tabelas do Censo 2010: 565325


---

### Spatial Join - Ponto em polígono

Realiza operações para assegurar que as linhas de logradouros pertencentes a um código regional (IBGE) interceptam o polígono desse código regional. 

In [7]:
# Import de novas bibliotecas:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon
from shapely.geometry.polygon import LinearRing
from shapely.geometry.linestring import LineString
import ast
import geopandas
from geopandas import GeoDataFrame

In [28]:
# Define funcao de correcao, necessaria devido ao tratamento feito no arquivo origem:
def corrige_ponto(shape):
    try:
        return Point(shape)
    except:
        try:
            return Point(shape[0])
        except:
            return Point(shape[0][0])

In [5]:
# Define funcao de correcao, necessaria devido ao tratamento feito no arquivo origem:
def corrige_linha(shape):
    try:
        return LineString(shape)
    except:
        try:
            return LineString(shape[0])
        except:
            return LineString(shape[0][0])

In [6]:
# Define funcao de correcao, necessaria devido ao tratamento feito no arquivo origem:
def corrige_poligono(shape):
    try:
        return Polygon(shape)
    except:
        try:
            return Polygon(shape[0])
        except:
            return Polygon(shape[0][0])

In [8]:
# Cria dataframe com os poligonos dos Codigos Regionais, por municipio:
arquivo_area = "data/shapefiles/tratados/areas_municipios.csv"
df_area = pd.read_csv(arquivo_area, delimiter="|", header=0, converters={"geometry.coordinates": ast.literal_eval})
df_area['coordenadas'] = df_area['geometry.coordinates']\
    .apply(lambda x: corrige_poligono(x))

In [11]:
df_area['coordenadas'].head()

0    POLYGON Z ((-48.23248007099994 -15.78359663799...
1    POLYGON Z ((-48.86700312799996 -15.09734690599...
2    POLYGON Z ((-46.97635062799998 -14.93112359199...
3    POLYGON Z ((-49.90713464099997 -17.61460280199...
4    POLYGON Z ((-48.58274617899997 -16.94785546499...
Name: coordenadas, dtype: object

In [12]:
# Cria geodataframe a partir do dataframe recem criado:
gdf_area = GeoDataFrame(
    df_area,
    geometry='coordenadas').drop('geometry.coordinates', axis=1)

In [13]:
gdf_area.head()

,properties.NOME,properties.CD_GEOCODI,properties.UF,coordenadas
0,BRASILIA,5300108,53,"POLYGON Z ((-48.23248 -15.78360 0.00000, -48.2..."
1,VILA PROPICIO,5222302,52,"POLYGON Z ((-48.86700 -15.09735 0.00000, -48.8..."
2,VILA BOA,5222203,52,"POLYGON Z ((-46.97635 -14.93112 0.00000, -46.9..."
3,VICENTINOPOLIS,5222054,52,"POLYGON Z ((-49.90713 -17.61460 0.00000, -49.9..."
4,VIANOPOLIS,5222005,52,"POLYGON Z ((-48.58275 -16.94786 0.00000, -48.5..."


In [31]:
# Realiza Spatial Join, gravando novos Shapefiles:
regioes = ['centro-oeste', 'nordeste', 'norte', 'sudeste', 'sul']
tipos_shape = ['roads', 'places', 'pois']
for regiao in regioes:
    gdf_area_regiao = gdf_area[gdf_area['properties.UF'].isin(list(df_ufs[df_ufs['regiao']==regiao]['codigo']))]
    for tipo_shape in tipos_shape:
        arquivo_origem = "data/shapefiles/tratados/{0}_{1}.csv"\
            .format(regiao, tipo_shape)
        arquivo_destino = "data/shapefiles/tratados/finais/{0}_{1}.shp"\
            .format(regiao, tipo_shape)
        print("\nImportando arquivo do tipo {0} da região '{1}'...".format(tipo_shape, regiao))
        df_regiao = pd.read_csv(arquivo_origem, delimiter="|", header=0, \
                                converters={"geometry.coordinates": ast.literal_eval})
        print("Iniciando tratamento...")
        try:
            df_regiao['coordenadas'] = df_regiao['geometry.coordinates']\
                .apply(lambda x: corrige_linha(x))
        except:
            df_regiao['coordenadas'] = df_regiao['geometry.coordinates']\
                .apply(lambda x: corrige_ponto(x))
        gdf_regiao = GeoDataFrame(
            df_regiao,
            geometry='coordenadas').drop('geometry.coordinates', axis=1)
        print("Realizando spatial join...")
        area_regiao = geopandas.sjoin(gdf_regiao, gdf_area_regiao, how="inner", op='intersects')
        print("Escrevendo arquivo {0}...".format(arquivo_destino))
        area_regiao.to_file(arquivo_destino)

print("\nFim das extrações.")


Importando arquivo do tipo roads da região 'centro-oeste'...
Iniciando tratamento...
Realizando spatial join...
Escrevendo arquivo C:/Users/Luiz/Documents/Python/geo/data/shapefiles/tratados/finais/centro-oeste_roads.shp...

Importando arquivo do tipo places da região 'centro-oeste'...
Iniciando tratamento...
Realizando spatial join...
Escrevendo arquivo C:/Users/Luiz/Documents/Python/geo/data/shapefiles/tratados/finais/centro-oeste_places.shp...

Importando arquivo do tipo pois da região 'centro-oeste'...
Iniciando tratamento...
Realizando spatial join...
Escrevendo arquivo C:/Users/Luiz/Documents/Python/geo/data/shapefiles/tratados/finais/centro-oeste_pois.shp...

Importando arquivo do tipo roads da região 'nordeste'...
Iniciando tratamento...
Realizando spatial join...
Escrevendo arquivo C:/Users/Luiz/Documents/Python/geo/data/shapefiles/tratados/finais/nordeste_roads.shp...

Importando arquivo do tipo places da região 'nordeste'...
Iniciando tratamento...
Realizando spatial join..

In [32]:
# Exemplo de resultado:
area_regiao.head()

,properties.name,properties.fclass,properties.ref,coordenadas,index_right,properties.NOME,properties.CD_GEOCODI,properties.UF
0,SUPERMERCADO CONDOR,SUPERMARKET,NaN,POINT (-49.24488 -25.43421),1564,CURITIBA,4106902,41
1,IBIS,HOTEL,NaN,POINT (-49.27118 -25.42446),1564,CURITIBA,4106902,41
2,MCDONALD'S,FAST_FOOD,NaN,POINT (-49.23518 -25.43599),1564,CURITIBA,4106902,41
3,RESTAURANTE BOA MESA,RESTAURANT,NaN,POINT (-49.23591 -25.43872),1564,CURITIBA,4106902,41
4,RESTAURANTE MILLONS,RESTAURANT,NaN,POINT (-49.23577 -25.43891),1564,CURITIBA,4106902,41


---

### Cruzamento dos novos Shapefiles com a base de endereços

In [119]:
# Import de biblioteca de regex:
import re

In [120]:
# Criacao de dicionario de numeros entre 0 e 100, para posterior aplicacao em regex:
lista_ate_cem = ['ZERO']
lista_1_9 = ['UM', 'DOIS', 'TRES', 'QUATRO', 'CINCO', 'SEIS', 'SETE', 'OITO', 'NOVE']
lista_10_19 = ['DEZ', 'ONZE', 'DOZE', 'TREZE', 'QUATORZE', 'QUINZE', 'DEZESSEIS', 'DEZESSETE', 'DEZOITO', 'DEZENOVE']
lista_dezenas = ['VINTE', 'TRINTA', 'QUARENTA', 'CINQUENTA', 'SESSENTA', 'SETENTA', 'OITENTA', 'NOVENTA', 'CEM']

lista_ate_cem += lista_1_9
lista_ate_cem += lista_10_19
dezena = 0
for n in range(20, 101):
    valor = ''
    if (n%10 == 0):
        dezena = int(n/10)-2
        valor = lista_dezenas[dezena]
    else:
        valor = lista_dezenas[dezena] + ' E ' + lista_1_9[(n%10)-1]
    lista_ate_cem.append(valor)

dict_ate_cem = {}
for i, n in enumerate(reversed(lista_ate_cem)):
    dict_ate_cem[n] = str(100-i)

dict_ate_cem['CATORZE'] = str(14)

dict_ate_cem['PRIMEIRA'] = str(1)
dict_ate_cem['SEGUNDA'] = str(2)
dict_ate_cem['TERCEIRA'] = str(3)
dict_ate_cem['QUARTA'] = str(4)
dict_ate_cem['QUINTA'] = str(5)
dict_ate_cem['SEXTA'] = str(6)
dict_ate_cem['SETIMA'] = str(7)
dict_ate_cem['OITAVA'] = str(8)
dict_ate_cem['NONA'] = str(9)
dict_ate_cem['DECIMA'] = str(10)
dict_ate_cem['VIGESIMA'] = str(20)

dict_ate_cem['PRIMEIRO'] = str(1)
dict_ate_cem['SEGUNDO'] = str(2)
dict_ate_cem['TERCEIRO'] = str(3)
dict_ate_cem['QUARTO'] = str(4)
dict_ate_cem['QUINTO'] = str(5)
dict_ate_cem['SEXTO'] = str(6)
dict_ate_cem['SETIMO'] = str(7)
dict_ate_cem['OITAVO'] = str(8)
dict_ate_cem['NONO'] = str(9)
dict_ate_cem['DECIMO'] = str(10)
dict_ate_cem['VIGESIMO'] = str(20)

rep = dict_ate_cem
rep = dict(('(^|\s)'+k+'($|\s)', ' ' + v + ' ') for k, v in rep.items())
pattern = re.compile("|".join(rep.keys()))

str(rep)

"{'(^|\\\\s)CEM($|\\\\s)': ' 100 ', '(^|\\\\s)NOVENTA E NOVE($|\\\\s)': ' 99 ', '(^|\\\\s)NOVENTA E OITO($|\\\\s)': ' 98 ', '(^|\\\\s)NOVENTA E SETE($|\\\\s)': ' 97 ', '(^|\\\\s)NOVENTA E SEIS($|\\\\s)': ' 96 ', '(^|\\\\s)NOVENTA E CINCO($|\\\\s)': ' 95 ', '(^|\\\\s)NOVENTA E QUATRO($|\\\\s)': ' 94 ', '(^|\\\\s)NOVENTA E TRES($|\\\\s)': ' 93 ', '(^|\\\\s)NOVENTA E DOIS($|\\\\s)': ' 92 ', '(^|\\\\s)NOVENTA E UM($|\\\\s)': ' 91 ', '(^|\\\\s)NOVENTA($|\\\\s)': ' 90 ', '(^|\\\\s)OITENTA E NOVE($|\\\\s)': ' 89 ', '(^|\\\\s)OITENTA E OITO($|\\\\s)': ' 88 ', '(^|\\\\s)OITENTA E SETE($|\\\\s)': ' 87 ', '(^|\\\\s)OITENTA E SEIS($|\\\\s)': ' 86 ', '(^|\\\\s)OITENTA E CINCO($|\\\\s)': ' 85 ', '(^|\\\\s)OITENTA E QUATRO($|\\\\s)': ' 84 ', '(^|\\\\s)OITENTA E TRES($|\\\\s)': ' 83 ', '(^|\\\\s)OITENTA E DOIS($|\\\\s)': ' 82 ', '(^|\\\\s)OITENTA E UM($|\\\\s)': ' 81 ', '(^|\\\\s)OITENTA($|\\\\s)': ' 80 ', '(^|\\\\s)SETENTA E NOVE($|\\\\s)': ' 79 ', '(^|\\\\s)SETENTA E OITO($|\\\\s)': ' 78 ', '(^|\\\\

In [121]:
# Exemplo de aplicacao da regex utilizando o dicionario de numeros recem criado:
frase = "RUA VINTE E DOIS"
pattern.sub(lambda m: rep['(^|\s)'+(m.group(0)).strip()+'($|\s)'], str(frase))

'RUA 22 '

In [122]:
# Definicao de uma lista de expressoes regulares a serem aplicadas, por ordem de indice:
palavras_exclusao = [r'^RUA ', r'^R ', r'^AVENIDA ', r'^AV ', 
                     r'^TRAVESSA ', r'^TRAV ', r'^BECO ', 
                     r'^ALAMEDA ', r'^AL ', r'^RODOVIA ', r'^VIA ',
                     r' DE ', r' DO ', r' DA ', r' DOS ', r' DAS ',
                     r'^DE ', r'^DO ', r'^DA ', r'^DOS ', r'^DAS ',
                     r'^SAO ', r'^S ', r'^SANTO ', r'^SANTA ', r'^STO ', r'^STA ',
                     r' LOTE .*', r' LT .*', r' APARTAMENTO .*', r' APTO .*', 
                     r' LOJA .*', r' LJ .*', r' ANTIGA .*',
                     r' CONJUNTO .*', r' CONJ .*',
                     r'^CORONEL ', r'^CEL ', r'^DOUTOR ', r'^DR ',
                     r'^DOUTORA ', r'^DRA ', 
                     r'^PROFESSOR ', r'^PROF ', r'^PROFESSORA ', r'^PROFA ',
                     r'^GOVERNADOR ', r'^SENADOR ', r'^MINISTRO ',
                     r'^PREFEITO ', r'^PRESIDENTE ',
                     r'^GENERAL ', r'^GEN ',
                     r'.*(?=(\s\w+){4})', r'.*(?=(\s\w+){3})', r'.*(?=(\s\w+){2})'
                    ]


In [123]:
# Definicao de funcao para aplicacao das expressoes regulares no geodataframe e no dataframe origens, 
#  seguida de join entre eles:
def aplica_regex_join(gdf, df, codigo_uf, variavel_origem, variavel_destino, regex, join='inner'):
    gdf[variavel_destino] = gdf[variavel_origem].apply(lambda x: re.sub('\s+', ' ', regex.sub(" ", str(x))).strip())
    
    df[variavel_destino] = df[variavel_origem].apply(lambda x: re.sub('\s+', ' ', regex.sub(" ", str(x))).strip())

    final_uf = gdf.merge(df, on=[variavel_destino, 'geocodigo'], how=join)
    return final_uf


In [124]:
# Percorre o geodataframe e o dataframe origens, gravando arquivo ao final:
def grava_arquivo_geo_CEP(arquivo_origem_gdf, arquivo_origem_df, destino, gravar=True, append=True):
    gdf = geopandas.read_file(arquivo_origem_gdf).drop_duplicates()
    gdf.columns = ['nome', 'tipo', 'nome_alternativo', 'index_right', 'municipio', 'geocodigo', 'codigo_uf', 'coordenadas']

    df = pd.read_csv(arquivo_origem_df, sep = '|').drop_duplicates()
    df.columns = ['geocodigo', 'nome', 'cep']
    df['geocodigo'] = df['geocodigo'].apply(lambda x: int(str(x)[0:7]))

    # Primeiros tratamentos:
    variavel_origem = 'nome'
    variavel_destino = 'nome2'
    gdf_origem = gdf.copy()
    gdf_origem[variavel_destino] = gdf_origem[variavel_origem]\
        .apply(lambda x: re.sub('\s+', ' ', re.sub("[\.\[\]\(\)\,\;\/]", " ", str(x))).strip())
    df_origem = df[df['cep'] >= 1000000].copy()
    df_origem['cep'] = df_origem['cep'].astype('int').astype('str')
    df_origem[variavel_destino] = df_origem[variavel_origem]\
        .apply(lambda x: re.sub('\s+', ' ', re.sub("[\.\[\]\(\)\,\;\/]", " ", str(x))).strip())
    aux_variavel_origem = variavel_origem

    # Substitui numeros:
    print('Tratando números...')
    gdf_origem[variavel_destino] = gdf_origem[variavel_destino]\
        .apply(lambda x: pattern.sub(lambda m: rep['(^|\s)'+(m.group(0)).strip()+'($|\s)'], str(x)))
    df_origem[variavel_destino] = df_origem[variavel_destino]\
        .apply(lambda x: pattern.sub(lambda m: rep['(^|\s)'+(m.group(0)).strip()+'($|\s)'], str(x)))

    # Mais tratamentos:
    #  - Tirar pontos dict_ate_cem['[\.\[\]\(\)\,\;\/]'] = ' '
    #  - Strip e tirar mais de um espaco seguido
    #  - Manter a coluna nome original
    #  - Trocar nomes de numeros por numeros
    #  - Um CEP encontrado não deve ser procurado novamente
    codigo_uf = int(str(df_origem['geocodigo'][0])[0:2])
    print('Código da UF: {0}.'.format(codigo_uf))
    array_ceps_restantes = df_origem['cep'].unique()
    n_ceps_origem = len(array_ceps_restantes)
    iniciado = False
    n_match_total = 0
    n_match_atual = 0
    n_match_anterior = 0
    palavras_excluidas = []
    variavel_origem = 'nome2'
    variavel_destino = 'nome3'
    colunas_df_origem = df_origem.columns
    df_origem['frequencia'] = df_origem.groupby(['nome', 'geocodigo', 'cep'])['cep'].transform('size')
    df_origem['maior_freq'] = df_origem.groupby(['cep'])['frequencia'].transform('max')
    aux_df_origem = df_origem[df_origem['frequencia']==df_origem['maior_freq']].drop_duplicates().copy()
    aux_df_origem.drop(['frequencia', 'maior_freq'], axis=1, inplace=True)
    aux_gdf_origem = gdf_origem[gdf_origem['codigo_uf']==codigo_uf].drop_duplicates().copy()
    progresso_palavras = 0
    progresso_cep = 0
    for p in palavras_exclusao:
        regex_excluir = re.compile(p)

        aux_df_final = aplica_regex_join(aux_gdf_origem, aux_df_origem,\
                                       codigo_uf, variavel_origem,\
                                       variavel_destino, regex_excluir)\
                            .drop_duplicates()

        aux_df_final['tratamento_final'] = p
        
        if (iniciado == False):
            df_final = aux_df_final.copy()
            iniciado = True
        else:
            df_final = df_final.append(aux_df_final, sort=False)

        n_match_atual = len(aux_df_final['cep'].unique())
        n_match_total = len(df_final['cep'].unique())
        if (n_match_atual > 0):
            n_match_anterior = n_match_atual
            palavras_excluidas.append(p + ': ' + str(n_match_atual))
            array_ceps_restantes = aux_df_origem[~aux_df_origem['cep'].isin(df_final['cep'].unique())]['cep'].unique()
            aux_df_origem = aux_df_origem[aux_df_origem['cep'].isin(array_ceps_restantes)]
        progresso_palavras = round(100*(palavras_exclusao.index(p))/len(palavras_exclusao), 2)
        progresso_cep = round(100*n_match_total/n_ceps_origem, 2)
        print('{0}% concluído ({1}) - Percentual de CEPs encontrados: {2}%              \r'\
              .format(progresso_palavras, p, progresso_cep), end="")
        variavel_origem = variavel_destino
    df_final.drop([i for i in list(df_final.columns) if '_y' in i], axis=1, inplace=True)
    df_final.drop(['index_right'], axis=1, inplace=True)
    df_final.columns = [re.sub('_x$', '', i) for i in list(df_final.columns)]
    df_final.drop_duplicates(inplace=True)
    df_final.groupby(['nome', 'geocodigo', 'cep'])['codigo_uf'].transform('size')
    df_final['frequencia'] = df_final.groupby(['nome', 'geocodigo', 'cep'])['codigo_uf'].transform('size')
    df_final['confianca_por_frequencia'] = round(df_final.groupby(['nome', 'geocodigo', 'cep'])['codigo_uf'].transform('size')\
                                  / df_final.groupby(['cep'])['frequencia'].transform('max'), 2)
    df_final['indice_verossimil'] = \
            df_final[variavel_destino].str.len()/df_final[aux_variavel_origem].str.len()
    progresso_palavras = round(100*(palavras_exclusao.index(p))/len(palavras_exclusao), 2)
    progresso_cep = round(100*max(n_match_atual, n_match_anterior)/n_ceps_origem, 2)
    print('Filtros aplicados, em ordem de mais influentes: {0}'\
          .format(', '.join(sorted(palavras_excluidas, reverse=True, key=lambda item: int(item.split(':')[1])))))
    print('\nPercentual de CEPs encontrados: {0}% ({1} de {2})'\
          .format(round(100*n_match_total/n_ceps_origem, 2),\
                 n_match_total, n_ceps_origem))
    if (gravar):
        if (append):
            df_final.to_csv(destino, mode='a+', header=False)
        else:
            df_final.to_csv(destino, mode='w+', header=True)
    else:
        return df_final


In [128]:
# Executa loop por ufs, tratando e exibindo percentual de CEPs idenficados em cada uma:
arquivo_destino = "data/resultados/geo_cep_202007.csv"
iniciado = False
gravar = True
ufs_gravacao = df_ufs
for index, uf in ufs_gravacao.iterrows():
    print("Carregando UF {0}...\r".format(uf['sigla']))
    arquivo_origem_gdf = "data/shapefiles/tratados/finais/{0}_roads.shp"\
        .format(uf['regiao'])
    arquivo_origem_df = "data/censo2010/enderecos_fins_estatisticos/tratados/{0}.csv"\
        .format(uf['sigla'])
    print("Tratando UF {0}...\r".format(uf['sigla']))
    if (gravar):
        grava_arquivo_geo_CEP(arquivo_origem_gdf, arquivo_origem_df, arquivo_destino, gravar, append=iniciado)
        print("UF {0} gravada no arquivo {1}.\n".format(uf['sigla'], arquivo_destino))
    else:
        aux_df_destino = grava_arquivo_geo_CEP(arquivo_origem_gdf, arquivo_origem_df, None, gravar, append=iniciado)
        if (iniciado):
            df_destino = df_destino.append(aux_df_destino, sort=False)
        else:
            df_destino = aux_df_destino.copy()
        print("UF {0} inserido no Dataframe {1}.\n".format(uf['sigla'], df_destino))
    iniciado = True


Carregando UF RO...
Tratando UF RO...
Tratando números...
Código da UF: 11.
Filtros aplicados, em ordem de mais influentes: ^RUA : 3742, ^AVENIDA : 592, ^TRAVESSA : 66, .*(?=(\s\w+){2}): 62, ^AV : 61, ^SAO : 59, ^PRESIDENTE : 29,  DE : 25, ^SANTA : 22, ^ALAMEDA : 19, ^DO : 19, ^DOUTOR : 18, ^GOVERNADOR : 15, ^BECO : 11, ^SANTO : 11,  DOS : 9, ^DAS : 9, .*(?=(\s\w+){3}): 9, ^DA : 8, ^DOS : 7,  DA : 6, ^CORONEL : 6, ^SENADOR : 6,  DO : 5, ^GENERAL : 4, ^R : 2, ^PROFESSOR : 2, ^DR : 1

Percentual de CEPs encontrados: 86.75% (4825 de 5562)
UF RO gravada no arquivo C:/Users/Luiz/Documents/Python/geo/data/resultados/geo_cep_202007.csv.

Carregando UF AC...
Tratando UF AC...
Tratando números...
Código da UF: 12.
Filtros aplicados, em ordem de mais influentes: ^RUA : 468, ^TRAVESSA : 43, ^AVENIDA : 23, ^SAO : 19, ^ALAMEDA : 11, .*(?=(\s\w+){2}): 9, ^SANTA : 8, ^CORONEL : 5, ^DA : 3, ^BECO : 2, .*(?=(\s\w+){3}): 2,  DO : 1, ^DE : 1, ^DO : 1, ^SANTO : 1, ^DOUTOR : 1, ^SENADOR : 1

Percentual de 

UF PE gravada no arquivo C:/Users/Luiz/Documents/Python/geo/data/resultados/geo_cep_202007.csv.

Carregando UF AL...
Tratando UF AL...
Tratando números...
Código da UF: 27.
Filtros aplicados, em ordem de mais influentes: ^RUA : 2631, .*(?=(\s\w+){2}): 309, ^AVENIDA : 154, ^SAO : 140, ^SANTA : 104, .*(?=(\s\w+){3}): 98, ^TRAVESSA : 96, ^DOUTOR : 93, ^PROFESSOR : 56, ^CORONEL : 32, ^SANTO : 31,  DE : 29, ^GOVERNADOR : 23, ^ALAMEDA : 21, ^DO : 17, ^PROFESSORA : 15, ^SENADOR : 11, ^PREFEITO : 10, ^PRESIDENTE : 10,  DA : 9, .*(?=(\s\w+){4}): 8, ^DOUTORA : 7, ^BECO : 6, ^DA : 5, ^GENERAL : 5, ^DE : 4, ^MINISTRO : 3,  DOS : 2, ^DAS : 2,  DO : 1, ^DOS : 1,  CONJ .*: 1

Percentual de CEPs encontrados: 83.49% (3934 de 4712)
UF AL gravada no arquivo C:/Users/Luiz/Documents/Python/geo/data/resultados/geo_cep_202007.csv.

Carregando UF SE...
Tratando UF SE...
Tratando números...
Código da UF: 28.
Filtros aplicados, em ordem de mais influentes: ^RUA : 1189, ^AVENIDA : 125, .*(?=(\s\w+){2}): 120, ^TR

Tratando números...
Código da UF: 50.
Filtros aplicados, em ordem de mais influentes: ^RUA : 5751, ^TRAVESSA : 359, ^AVENIDA : 335, .*(?=(\s\w+){2}): 226, ^SAO : 80, ^ALAMEDA : 75, ^DOUTOR : 69, ^SANTA : 49,  DE : 42, .*(?=(\s\w+){3}): 42, ^PRESIDENTE : 41, ^GENERAL : 39, ^CORONEL : 36, ^SANTO : 20, ^DO : 19, ^DA : 15, ^PROFESSOR : 15, ^SENADOR : 13,  DA : 12, ^DE : 11,  DO : 7, ^DOS : 7, ^PROFESSORA : 6, ^MINISTRO : 6, ^DAS : 3, ^R : 2, ^RODOVIA : 1,  DOS : 1, ^STA : 1, ^DR : 1, ^DOUTORA : 1, ^PREFEITO : 1, .*(?=(\s\w+){4}): 1

Percentual de CEPs encontrados: 87.23% (7287 de 8354)
UF MS gravada no arquivo C:/Users/Luiz/Documents/Python/geo/data/resultados/geo_cep_202007.csv.

Carregando UF MT...
Tratando UF MT...
Tratando números...
Código da UF: 51.
Filtros aplicados, em ordem de mais influentes: ^RUA : 3764, ^AVENIDA : 519, .*(?=(\s\w+){2}): 135, ^TRAVESSA : 45, ^SAO : 41, ^R : 35, ^PRESIDENTE : 34, .*(?=(\s\w+){3}): 25, ^CORONEL : 21,  DE : 18, ^SANTA : 16, ^GOVERNADOR : 15, ^PROFE

---

### Busca por locais próximos e CEP relacionado ao respectivo centroide

In [94]:
# Import de novas bibliotecas:
from shapely import wkt
from shapely.geometry import MultiPoint
from shapely.ops import nearest_points
import geopy.distance as dist
from scipy.spatial import distance
import numpy as np

In [241]:
# Define funcao que retorna ponto proximo a poligono:
def get_proximo(ponto, pts=pts3):
    return nearest_points(ponto, pts)[1]

In [210]:
# Define funcao que retorna distancia entre dois pontos:
def distancia_dois_pontos(p1, p2):
    return dist.distance((p1.y, p1.x), (p2.y, p2.x)).km

In [212]:
# Define funcao que retorna centroide a partir de tres pontos:
def centroide_tres_pontos(p1, p2, p3):
    pontos = MultiPoint([(p1.x, p1.y), (p2.x, p2.y), (p3.x, p3.y)])
    return pontos.centroid

In [252]:
# Define funcao que retorna o CEP mais proximo ao ponto informado:
def retorna_CEP(ponto, df_geocep, lista_geocodigos):
    gdf_geocep = GeoDataFrame(
        df_geocep[df_geocep['geocodigo'].isin(lista_geocodigos)],
        geometry='coordenadas')
    pts3 = gdf_geocep.geometry.unary_union
    return geopandas.sjoin(GeoDataFrame([get_proximo(ponto, pts3)], \
                                        columns=['geometry']), \
                           gdf_geocep, how="inner", op='intersects')

In [250]:
arquivo_geocep = "C:/Users/Luiz/Documents/Python/geo/data/resultados/geo_cep_202007.csv"
df_geocep = pd.read_csv(arquivo_geocep, delimiter=",", header=0)
df_geocep['coordenadas'] = df_geocep['coordenadas'].apply(wkt.loads)

In [279]:
# Define funcao que procura por locais proximos segundo a distancia maxima e filtro informados.
#  Devido a didatica aplicada, o filtro aqui era um divisor do numero de CEP, que era aplicado para se
#  observar se o resto da divisao do CEP por esse divisor era zero:
def procura_locais(gdf_locais, df_geocep, tipo_1, tipo_2, tipo_3, distancia_maxima_1, distancia_maxima_2, divisor):
    tamanho = len(gdf_locais[gdf_locais['tipo']==tipo_1])
    n = 0
    for i1, r1 in gdf_locais[gdf_locais['tipo']==tipo_1].iterrows():
        print("Progresso: {0}%          \r".format(round(100*n/tamanho)), end="")
        n += 1
        for i2, r2 in gdf_locais[gdf_locais['tipo']==tipo_2].iterrows():
            distancia_1 = distancia_dois_pontos(r1['coordenadas'], r2['coordenadas'])
            if (distancia_1 < distancia_maxima_1):
                for i3, r3 in gdf_locais[gdf_locais['tipo']==tipo_3].iterrows():
                    distancia_2 = distancia_dois_pontos(r2['coordenadas'], r3['coordenadas'])
                    if (distancia_2 < distancia_maxima_2):
                        print("{0} ({1}, {2}) está a {3} km de " \
                              "{4} ({5}, {6}), que está a {7} km de " \
                              "{8} ({9}, {10})."\
                              .format(r1['nome'], r1['coordenadas'].y, r1['coordenadas'].x,\
                                      str(round(distancia_1, 2)),\
                                      r2['nome'], r2['coordenadas'].y, r2['coordenadas'].x,\
                                      str(round(distancia_2, 2)),\
                                      r3['nome'], r3['coordenadas'].y, r3['coordenadas'].x\
                                     ))
                        aux_geocodigos = [r1['geocodigo'], r2['geocodigo'], r3['geocodigo']]
                        geocodigos = list(set(aux_geocodigos))
                        centroide = centroide_tres_pontos(r1['coordenadas'], r2['coordenadas'], r3['coordenadas'])
                        print("O centroide entre esses três pontos é ({0}, {1})"\
                             .format(centroide.y, centroide.x))
                        try:
                            cep = retorna_CEP(centroide, df_geocep, geocodigos)['cep'][0]
                            print("O CEP mais próximo do centroide é {0}, cujo resto da divisão por {1} é {2}"\
                                  .format(cep, divisor, cep%divisor))
                        except:
                            print("Nenhum CEP encontrado para o(s) geocódigos {0}"\
                              .format(geocodigos))
                        print("\n----------------------------------------------------\n")
                    
        

In [ ]:
# Exemplo de busca por locais onde um fast food esta a ate 100 metros de um hotel, 
#  o qual esta a ate 200 metros de uma livraria/banca:

tipo_1 = 'FAST_FOOD'
tipo_2 = 'HOTEL'
tipo_3 = 'BOOKSHOP'

distancia_maxima_1 = 0.1
distancia_maxima_2 = 0.2
divisor = 4

regioes = ['centro-oeste', 'nordeste', 'norte', 'sudeste', 'sul']

for r in regioes:
    print("\nPesquisando na região '{0}'...\n".format(r))
    arquivo_locais = "data/shapefiles/tratados/finais/{0}_pois.shp"\
        .format(r)
    gdf_locais = geopandas.read_file(arquivo_locais)
    gdf_locais.drop(['index_righ'], axis=1, inplace=True)
    gdf_locais.columns = ['nome', 'tipo', 'nome_alternativo', 'municipio', 'geocodigo', 'codigo_uf', 'coordenadas']
    procuraLocais(gdf_locais, df_geocep, tipo_1, tipo_2, tipo_3, distancia_maxima_1, distancia_maxima_2, divisor)
    
print("\nFim das pesquisas")


Pesquisando na região 'centro-oeste'...

LANCHES CENTRAL (-22.5398509, -55.7261391) está a 0.07 km de HOTEL DE TRANSITO E OFICIAIS (-22.5393163, -55.7264685), que está a 0.14 km de PROFETICA (-22.5382873, -55.7273028).
O centroide entre esses três pontos é (-22.539151500000003, -55.726636799999994)
Nenhum CEP encontrado para o(s) geocódigos [5006606]

----------------------------------------------------

FILHO DA FRUTA (-22.5390744, -55.7266452) está a 0.03 km de HOTEL DE TRANSITO E OFICIAIS (-22.5393163, -55.7264685), que está a 0.14 km de PROFETICA (-22.5382873, -55.7273028).
O centroide entre esses três pontos é (-22.53889266666667, -55.726805500000005)
Nenhum CEP encontrado para o(s) geocódigos [5006606]

----------------------------------------------------

SUBWAY (-22.5396069, -55.726312) está a 0.04 km de HOTEL DE TRANSITO E OFICIAIS (-22.5393163, -55.7264685), que está a 0.14 km de PROFETICA (-22.5382873, -55.7273028).
O centroide entre esses três pontos é (-22.539070166666665

LANCHONETE DO CEARA (-15.2454549, -40.2481106) está a 0.04 km de HOTEL DEL REY (-15.2455082, -40.2477811), que está a 0.16 km de REI DOS REIS (-15.2454055, -40.2463339).
O centroide entre esses três pontos é (-15.245456200000001, -40.24740853333333)
Nenhum CEP encontrado para o(s) geocódigos [2916401]

----------------------------------------------------

LANCHONETE DO CEARA (-15.2454549, -40.2481106) está a 0.09 km de ITAHOTEL (-15.2455611, -40.2472405), que está a 0.1 km de REI DOS REIS (-15.2454055, -40.2463339).
O centroide entre esses três pontos é (-15.245473833333335, -40.24722833333333)
Nenhum CEP encontrado para o(s) geocódigos [2916401]

----------------------------------------------------

LANCHONETE DO CEARA (-15.2454549, -40.2481106) está a 0.09 km de ITAHOTEL (-15.2455611, -40.2472405), que está a 0.18 km de CANTINHO DO CRISTAO (-15.2453589, -40.2455391).
O centroide entre esses três pontos é (-15.245458300000001, -40.2469634)
O CEP mais próximo do centroide é 45705000, c

Nenhum CEP encontrado para o(s) geocódigos [3106200]

----------------------------------------------------

MULINO PIZZERIA (-19.9390606, -43.9386621) está a 0.06 km de HOTEL BOULEVARD (-19.9393502, -43.9381793), que está a 0.12 km de LIVRARIA LEITURA (-19.9391445, -43.9370442).
O centroide entre esses três pontos é (-19.9391851, -43.93796186666666)
Nenhum CEP encontrado para o(s) geocódigos [3106200]

----------------------------------------------------

MULINO PIZZERIA (-19.9390606, -43.9386621) está a 0.06 km de HOTEL BOULEVARD (-19.9393502, -43.9381793), que está a 0.18 km de QUIXOTE LIVRARIA E CAFE (-19.939122, -43.9364426).
O centroide entre esses três pontos é (-19.939177599999997, -43.937761333333334)
Nenhum CEP encontrado para o(s) geocódigos [3106200]

----------------------------------------------------

MULINO PIZZERIA (-19.9390606, -43.9386621) está a 0.06 km de HOTEL BOULEVARD (-19.9393502, -43.9381793), que está a 0.17 km de ESQUERDA LITERARIA (-19.937897, -43.9376651).


Acredito ser necessário ajustar o tratamento dos geocódigos (Códigos Regionais/de Setores do IBGE), visto que foi recorrente a ausência de CEP em geocódigos. Além disso, a execução de três loops não é performática, certamente existindo opções melhores para substituí-los.

De qualquer forma, para o objetivo da apresentação esse tipo de correção não foi essencial e alcançou-se uma base de 425 mil CEPs distintos, com seus dados e polígonos correspondentes, além de 211 mil pontos de interesse.